In [0]:
#http://nicolas-hug.com/blog/matrix_facto_4
#https://nbviewer.jupyter.org/github/NicolasHug/nicolashug.github.io/blob/master/assets/mf_post/Matrix%20factorization%20algorithm.ipynb#

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive #toimport google drive data
drive.mount('/content/drive') #to mount the drive data into colab cloud and enterr the activation code

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/"My Drive"

/content/drive/My Drive


In [0]:
import zipfile
zipfile = zipfile.ZipFile('NG Machine Learning/movielensrecommendation.zip', 'r')
zipfile.extractall()
zipfile.close()

In [0]:
import zipfile
zipfile = zipfile.ZipFile('NG Machine Learning/movielensrecommendation.zip', 'r')


In [0]:
with open('./ml-100k/u.data') as f:
    all_lines = f.readlines()

In [11]:
pip install scikit-surprise

     |████████████████████████████████| 6.5MB 27.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678208 sha256=c8179b4a8fe0f0363e331a62b239bdb05a9ede66c61207bb441f296ff13d371d
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import zipfile
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [0]:
import surprise
# Define the format
reader = Reader(line_format='user item rating timestamp', sep='\t')
# Load the data from the file using the reader format
data = Dataset.load_from_file('./ml-100k/u.data', reader=reader)

In [18]:
#data.split(2) ## split data for 2-folds cross validation

AttributeError: ignored

In [0]:
#Matrix factorization with Stochastic Gradient Descent Algorithm
class MatrixFacto(surprise.AlgoBase):
    '''A basic rating prediction algorithm based on matrix factorization.'''
    
    def __init__(self, learning_rate, n_epochs, n_factors):
        
        self.lr = learning_rate  # learning rate for SGD
        self.n_epochs = n_epochs  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors
        
    def fit(self, trainset):
        '''Learn the vectors p_u and q_i with SGD'''
        
        print('Fitting data with SGD...')
        
        # Randomly initialize the user and item factors.
        p = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        q = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        
        # SGD procedure
        for _ in range(self.n_epochs):
            for u, i, r_ui in trainset.all_ratings():
                err = r_ui - np.dot(p[u], q[i])
                # Update vectors p_u and q_i
                p[u] += self.lr * err * q[i]
                q[i] += self.lr * err * p[u]
                # Note: in the update of q_i, we should actually use the previous (non-updated) value of p_u.
                # In practice it makes almost no difference.
        
        self.p, self.q = p, q
        self.trainset = trainset

    def estimate(self, u, i):
        '''Return the estmimated rating of user u for item i.'''
        
        # return scalar product between p_u and q_i if user and item are known,
        # else return the average of all ratings
        if self.trainset.knows_user(u) and self.trainset.knows_item(i):
            return np.dot(self.p[u], self.q[i])
        else:
            return self.trainset.global_mean

In [0]:
algo = MatrixFacto(learning_rate=.01, n_epochs=10, n_factors=10)
#surprise.evaluate(algo, data, measures=['RMSE'])

In [21]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Fitting data with SGD...
Fitting data with SGD...
Fitting data with SGD...
Fitting data with SGD...
Fitting data with SGD...
Evaluating RMSE, MAE of algorithm MatrixFacto on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9622  0.9605  0.9554  0.9581  0.9550  0.9582  0.0028  
MAE (testset)     0.7524  0.7541  0.7500  0.7525  0.7492  0.7516  0.0018  
Fit time          7.96    7.83    7.93    7.97    7.91    7.92    0.05    
Test time         0.24    0.16    0.17    0.23    0.16    0.19    0.03    


{'fit_time': (7.964536905288696,
  7.826776027679443,
  7.928162097930908,
  7.970513105392456,
  7.911818981170654),
 'test_mae': array([0.7523847 , 0.75407795, 0.74998804, 0.7525041 , 0.74923327]),
 'test_rmse': array([0.96221154, 0.96049424, 0.95538804, 0.95811273, 0.95501013]),
 'test_time': (0.2417893409729004,
  0.16097378730773926,
  0.17203593254089355,
  0.22514605522155762,
  0.1581130027770996)}

In [22]:
# try a neighborhood-based algorithm (on the same data)
algo = surprise.KNNBasic()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
#surprise.evaluate(algo, data, measures=['RMSE'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9719  0.9729  0.9851  0.9763  0.9839  0.9780  0.0055  
MAE (testset)     0.7680  0.7685  0.7763  0.7722  0.7774  0.7725  0.0039  
Fit time          0.38    0.46    0.50    0.38    0.38    0.42    0.05    
Test time         3.99    4.23    3.90    3.74    3.86    3.94    0.17    


{'fit_time': (0.3794243335723877,
  0.4579319953918457,
  0.4986135959625244,
  0.38092041015625,
  0.37808871269226074),
 'test_mae': array([0.7680022 , 0.76853984, 0.77631013, 0.77224727, 0.77742594]),
 'test_rmse': array([0.97188119, 0.97292111, 0.98506199, 0.97627668, 0.9838833 ]),
 'test_time': (3.9901368618011475,
  4.234817266464233,
  3.896660327911377,
  3.7391321659088135,
  3.857728958129883)}

In [23]:
algo = surprise.SVD()
#surprise.evaluate(algo, data, measures=['RMSE'])
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9262  0.9381  0.9374  0.9433  0.9353  0.9361  0.0056  
MAE (testset)     0.7325  0.7372  0.7385  0.7447  0.7394  0.7385  0.0039  
Fit time          5.21    5.04    5.15    5.14    5.27    5.16    0.08    
Test time         0.15    0.23    0.15    0.23    0.15    0.18    0.04    


{'fit_time': (5.21277642250061,
  5.040224075317383,
  5.152867078781128,
  5.1371543407440186,
  5.266265153884888),
 'test_mae': array([0.73249387, 0.73721321, 0.73852346, 0.74474669, 0.7393524 ]),
 'test_rmse': array([0.92621299, 0.93810554, 0.93737752, 0.94327793, 0.93528045]),
 'test_time': (0.15432286262512207,
  0.22827601432800293,
  0.14520692825317383,
  0.22563767433166504,
  0.1518082618713379)}